In [24]:
### migrate code from tensorflow v1 to v2: 
# !tf_upgrade_v2 \
#   --infile class_DeepLongitudinal-Original.py \
#   --outfile class_DeepLongitudinal-Original_v2.py

# !tf_upgrade_v2 \
#   --infile utils_network-Original.py \
#   --outfile utils_network-Original_v2.py

In [1]:
_EPSILON = 1e-08

import numpy as np
import pandas as pd
import tensorflow as tf
import random
import os

from sklearn.model_selection import train_test_split

import import_data as impt
from tf_slim.layers import layers as _layers
from class_DeepLongitudinal import Model_Longitudinal_Attention

from utils_eval             import c_index, brier_score
from utils_log              import save_logging, load_logging
from utils_helper           import f_get_minibatch, f_get_boosted_trainset

In [2]:
def _f_get_pred(sess, model, data, data_mi, pred_horizon):
    '''
        predictions based on the prediction time.
        create new_data and new_mask2 that are available previous or equal to the prediction time (no future measurements are used)
    '''
    new_data    = np.zeros(np.shape(data))
    new_data_mi = np.zeros(np.shape(data_mi))

    meas_time = np.concatenate([np.zeros([np.shape(data)[0], 1]), np.cumsum(data[:, :, 0], axis=1)[:, :-1]], axis=1)

    for i in range(np.shape(data)[0]):
        last_meas = np.sum(meas_time[i, :] <= pred_horizon)

        new_data[i, :last_meas, :]    = data[i, :last_meas, :]
        new_data_mi[i, :last_meas, :] = data_mi[i, :last_meas, :]

    return model.predict(new_data, new_data_mi)


def f_get_risk_predictions(sess, model, data_, data_mi_, pred_time, eval_time):
    
    pred = _f_get_pred(sess, model, data_[[0]], data_mi_[[0]], 0)
    _, num_Event, num_Category = np.shape(pred)
       
    risk_all = {}
    for k in range(num_Event):
        risk_all[k] = np.zeros([np.shape(data_)[0], len(pred_time), len(eval_time)])
            
    for p, p_time in enumerate(pred_time):
        ### PREDICTION
        pred_horizon = int(p_time)
        pred = _f_get_pred(sess, model, data_, data_mi_, pred_horizon)


        for t, t_time in enumerate(eval_time):
            eval_horizon = int(t_time) + pred_horizon #if eval_horizon >= num_Category, output the maximum...

            # calculate F(t | x, Y, t >= t_M) = \sum_{t_M <= \tau < t} P(\tau | x, Y, \tau > t_M)
            risk = np.sum(pred[:,:,pred_horizon:(eval_horizon+1)], axis=2) #risk score until eval_time
            risk = risk / (np.sum(np.sum(pred[:,:,pred_horizon:], axis=2), axis=1, keepdims=True) +_EPSILON) #conditioniong on t > t_pred
            
            for k in range(num_Event):
                risk_all[k][:, p, t] = risk[:, k]
                
    return risk_all

In [3]:
### Set prediction time window (t) and evaluation time (delta t) for C-index and Brier-Score)
pred_time = list(range(16,33,1)) # prediction time (in months)
eval_time = list(range(1))  


### 1. Import Dataset
#####      - Users must prepare dataset in csv format and modify 'import_data.py' following our examplar 'PBC2'

In [4]:
df = pd.read_csv('./data/data_longi_ascvd_for_dynamic_deephit.csv')

trainingid_all = pd.read_csv('./data/all_training_set_ID.csv')
validationid_all = pd.read_csv('./data/all_validation_set_ID.csv')
testingid_all = pd.read_csv('./data/all_testing_set_ID.csv')



In [5]:
df

,ID,event,exam_year,time,AGE_Y0,MALE,RACEBLACK,CHOL,DIAB,HBM,HDL,SBP,SMKNW
0,100033323702,0,0,32.134155,22,1,0,231,0,0,42,117.0,1
1,100033323702,0,2,32.134155,22,1,0,187,0,0,46,116.0,1
2,100033323702,0,5,32.134155,22,1,0,234,0,0,52,105.0,0
3,100033323702,0,7,32.134155,22,1,0,216,0,0,44,115.0,0
4,100033323702,0,15,32.134155,22,1,0,214,0,0,42,107.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20201,416817227898,0,2,31.770021,22,1,1,203,0,0,56,102.0,1
20202,416817227898,0,5,31.770021,22,1,1,204,0,0,43,133.0,1
20203,416817227898,0,7,31.770021,22,1,1,170,0,0,47,133.0,1
20204,416817227898,0,10,31.770021,22,1,1,179,0,0,47,112.0,0


In [6]:
#data_mode                   = 'PBC2' 
data_mode                   = 'CARDIA_ASCVD' 
seed                        = 1234

##### IMPORT DATASET
'''
    num_Category            = max event/censoring time * 1.2
    num_Event               = number of evetns i.e. len(np.unique(label))-1
    max_length              = maximum number of measurements
    x_dim                   = data dimension including delta (1 + num_features)
    x_dim_cont              = dim of continuous features
    x_dim_bin               = dim of binary features
    mask1, mask2, mask3     = used for cause-specific network (FCNet structure)
'''

# (x_dim, x_dim_cont, x_dim_bin), (data, time, label), (mask1, mask2, mask3), (data_mi) = impt.import_dataset(norm_mode = 'standard')



(x_dim, x_dim_cont, x_dim_bin), (data, time, label), (mask1, mask2, mask3), (data_mi) = impt.import_dataset(df_ = df)





_, num_Event, num_Category  = np.shape(mask1)  # dim of mask3: [subj, Num_Event, Num_Category]
max_length                  = np.shape(data)[1]


file_path = '{}'.format(data_mode)

if not os.path.exists(file_path):
    os.makedirs(file_path)

In [9]:
data.shape

(3551, 6, 10)

In [11]:
mask1.shape

(3551, 1, 39)

In [14]:
mask3.shape

(3551, 39)

In [12]:
data_mi.shape

(3551, 6, 10)

In [18]:
x_dim_bin

5

### 2. Set Hyper-Parameters
##### - Play with your own hyper-parameters!

In [7]:
burn_in_mode                = 'ON' #{'ON', 'OFF'}
boost_mode                  = 'ON' #{'ON', 'OFF'}

##### HYPER-PARAMETERS
new_parser = {'mb_size': 32,

             'iteration_burn_in': 3000,
             'iteration': 25000,

             'keep_prob': 0.6,
             'lr_train': 1e-4,

             'h_dim_RNN': 100,
             'h_dim_FC' : 100,
             'num_layers_RNN':2,
             'num_layers_ATT':2,
             'num_layers_CS' :2,

             'RNN_type':'LSTM', #{'LSTM', 'GRU'}

             'FC_active_fn' : tf.nn.relu,
             'RNN_active_fn': tf.nn.tanh,

            'reg_W'         : 1e-5,
            'reg_W_out'     : 0.,

             'alpha' :1.0,
             'beta'  :0.1,
             'gamma' :1.0
}


# INPUT DIMENSIONS
input_dims                  = { 'x_dim'         : x_dim,
                                'x_dim_cont'    : x_dim_cont,
                                'x_dim_bin'     : x_dim_bin,
                                'num_Event'     : num_Event,
                                'num_Category'  : num_Category,
                                'max_length'    : max_length }

# NETWORK HYPER-PARMETERS
network_settings            = { 'h_dim_RNN'         : new_parser['h_dim_RNN'],
                                'h_dim_FC'          : new_parser['h_dim_FC'],
                                'num_layers_RNN'    : new_parser['num_layers_RNN'],
                                'num_layers_ATT'    : new_parser['num_layers_ATT'],
                                'num_layers_CS'     : new_parser['num_layers_CS'],
                                'RNN_type'          : new_parser['RNN_type'],
                                'FC_active_fn'      : new_parser['FC_active_fn'],
                                'RNN_active_fn'     : new_parser['RNN_active_fn'],
                               # 'initial_W'         : tf.contrib.layers.xavier_initializer(),
                               
                                'initial_W'         : tf.keras.initializers.glorot_normal(),

                               
                                'reg_W'             : new_parser['reg_W'],
                                'reg_W_out'         : new_parser['reg_W_out']
                                 }


mb_size           = new_parser['mb_size']
iteration         = new_parser['iteration']
iteration_burn_in = new_parser['iteration_burn_in']

keep_prob         = new_parser['keep_prob']
lr_train          = new_parser['lr_train']

alpha             = new_parser['alpha']
beta              = new_parser['beta']
gamma             = new_parser['gamma']

# SAVE HYPERPARAMETERS
log_name = file_path + '/hyperparameters_log.txt'
save_logging(new_parser, log_name)

### 4. Train the Network

In [8]:
## Trained with number of iteration 25000 -> 50000

fold = 1
print('FOLD '+str(fold) + '...')

##### get training, testing, and validation data:
df_train = df.loc[df['ID'].isin(trainingid_all.iloc[:,fold])]
df_val = df.loc[df['ID'].isin(validationid_all.iloc[:,fold])]
df_test = df.loc[df['ID'].isin(testingid_all.iloc[:,fold])]

# ### TRAINING-TESTING SPLIT in the format suitable for this network

(x_dim, x_dim_cont, x_dim_bin), (te_data, te_time, te_label), (te_mask1, te_mask2, te_mask3), (te_data_mi) = impt.import_dataset(df_ = df_test)
(x_dim, x_dim_cont, x_dim_bin), (va_data, va_time, va_label), (va_mask1, va_mask2, va_mask3), (va_data_mi) = impt.import_dataset(df_ = df_val)
(x_dim, x_dim_cont, x_dim_bin), (tr_data, tr_time, tr_label), (tr_mask1, tr_mask2, tr_mask3), (tr_data_mi) = impt.import_dataset(df_ = df_train)

if boost_mode == 'ON':
    tr_data, tr_data_mi, tr_time, tr_label, tr_mask1, tr_mask2, tr_mask3 = f_get_boosted_trainset(tr_data, tr_data_mi, tr_time, tr_label, tr_mask1, tr_mask2, tr_mask3)  







##### CREATE AND TRAIN NETWORK:
# tf.reset_default_graph()
tf.compat.v1.reset_default_graph()

# config = tf.ConfigProto()
config = tf.compat.v1.ConfigProto()

config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)

model = Model_Longitudinal_Attention(sess, "Dyanmic-DeepHit", input_dims, network_settings)
# saver = tf.train.Saver()
saver = tf.compat.v1.train.Saver()

# sess.run(tf.global_variables_initializer())
sess.run(tf.compat.v1.global_variables_initializer())

### TRAINING - BURN-IN
if burn_in_mode == 'ON':
    print( "BURN-IN TRAINING ...")
    for itr in range(iteration_burn_in):
        x_mb, x_mi_mb, k_mb, t_mb, m1_mb, m2_mb, m3_mb = f_get_minibatch(mb_size, tr_data, tr_data_mi, tr_label, tr_time, tr_mask1, tr_mask2, tr_mask3)
        DATA = (x_mb, k_mb, t_mb)
        MISSING = (x_mi_mb)

        _, loss_curr = model.train_burn_in(DATA, MISSING, keep_prob, lr_train)

        if (itr+1)%1000 == 0:
            print('itr: {:04d} | loss: {:.4f}'.format(itr+1, loss_curr))


### TRAINING - MAIN
print( "MAIN TRAINING ...")
min_valid = 0.5

for itr in range(iteration):
    x_mb, x_mi_mb, k_mb, t_mb, m1_mb, m2_mb, m3_mb = f_get_minibatch(mb_size, tr_data, tr_data_mi, tr_label, tr_time, tr_mask1, tr_mask2, tr_mask3)
    DATA = (x_mb, k_mb, t_mb)
    MASK = (m1_mb, m2_mb, m3_mb)
    MISSING = (x_mi_mb)
    PARAMETERS = (alpha, beta, gamma)

    _, loss_curr = model.train(DATA, MASK, MISSING, PARAMETERS, keep_prob, lr_train)

    if (itr+1)%1000 == 0:
        print('itr: {:04d} | loss: {:.4f}'.format(itr+1, loss_curr))

    ### VALIDATION  (based on average C-index of our interest)
    if (itr+1)%1000 == 0:        
        risk_all = f_get_risk_predictions(sess, model, va_data, va_data_mi, pred_time, eval_time)

        for p, p_time in enumerate(pred_time):
            pred_horizon = int(p_time)
            val_result1 = np.zeros([num_Event, len(eval_time)])

            for t, t_time in enumerate(eval_time):                
                eval_horizon = int(t_time) + pred_horizon
                for k in range(num_Event):
                    val_result1[k, t] = c_index(risk_all[k][:, p, t], va_time, (va_label[:,0] == k+1).astype(int), eval_horizon) #-1 for no event (not comparable)

            if p == 0:
                val_final1 = val_result1
            else:
                val_final1 = np.append(val_final1, val_result1, axis=0)

        tmp_valid = np.mean(val_final1)

        if tmp_valid >  min_valid:
            min_valid = tmp_valid
            saver.save(sess, file_path + '/model')
            print( 'updated.... average c-index = ' + str('%.4f' %(tmp_valid)))








### PREDICTION ON TEST SET               
#saver.restore(sess, file_path + '/model')

risk_all = f_get_risk_predictions(sess, model, te_data, te_data_mi, pred_time, eval_time)

for p, p_time in enumerate(pred_time):
    pred_horizon = int(p_time)
    result1, result2 = np.zeros([num_Event, len(eval_time)]), np.zeros([num_Event, len(eval_time)])

    for t, t_time in enumerate(eval_time):                
        eval_horizon = int(t_time) + pred_horizon
        for k in range(num_Event):
            result1[k, t] = c_index(risk_all[k][:, p, t], te_time, (te_label[:,0] == k+1).astype(int), eval_horizon) #-1 for no event (not comparable)
            result2[k, t] = brier_score(risk_all[k][:, p, t], te_time, (te_label[:,0] == k+1).astype(int), eval_horizon) #-1 for no event (not comparable)

    if p == 0:
        final1, final2 = result1, result2
    else:
        final1, final2 = np.append(final1, result1, axis=0), np.append(final2, result2, axis=0)








### PRINT PERFORMANCE RESULTS
row_header = []
for p_time in pred_time:
    for t in range(num_Event):
        row_header.append('pred_time {}: event_{}'.format(p_time,k+1))

col_header = []
for t_time in eval_time:
    col_header.append('eval_time {}'.format(t_time))


# c-index result
df1 = pd.DataFrame(final1, index = row_header, columns=col_header)

# brier-score result
df2 = pd.DataFrame(final2, index = row_header, columns=col_header)

print('========================================================')
print('--------------------------------------------------------')
print('- C-INDEX: ')
print(df1)
print('--------------------------------------------------------')
print('- BRIER-SCORE: ')
print(df2)
print('========================================================')








### SAVE C-INDEX, BRIER SCORE, and PREDICTED PROB RISK ON TEST SET
actual_fold = fold+1
work_dir = 'U:/Hieu/CARDIA_longi_project'
savedir = os.path.join(work_dir,'rdata_files/dynamic_deephit_ascvd_var_y15_fold_'+str(actual_fold)+'/')
try: 
    os.makedirs(savedir)
except OSError:
    if not os.path.isdir(savedir):
        raise



c_over_time = df1.iloc[:,0]
# c_over_time.to_csv(savedir+'/c_index.csv', index = None, header = True)

brier_over_time = df2.iloc[:,0]
# brier_over_time.to_csv(savedir+'/brier_score.csv', index = None, header = True)



prob_risk_test_df = pd.DataFrame(risk_all[0][:,:,0])
prob_risk_test_df.columns = pred_time
prob_risk_test_df.insert(loc=0, column='ID', value=np.unique(df_test['ID']))
# prob_risk_test_df.to_csv(savedir+'/prob_risk_test.csv', index = None, header = True)



FOLD 1...
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


C:\Users\hnguye78\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\layers\legacy_rnn\rnn_cell_impl.py:903: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
C:\Users\hnguye78\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '
C:\Users\hnguye78\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1719: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
BURN-IN TRAINING ...
itr: 1000 | loss: 2.1242
itr: 2000 | loss: 1.8200
itr: 3000 | loss: 1.4469
MAIN TRAINING ...
itr: 1000 | loss: 2.0664
itr: 2000 | loss: 1.4569
updated.... average c-index = 0.5223
itr: 3000 | loss: 1.6083
updated.... average c-index = 0.5399
itr: 4000 | loss: 1.6314
itr: 5000 | loss: 1.3455
updated.... average c-index = 0.5401
itr: 6000 | loss: 1.8566
updated.... average c-index = 0.5472
itr: 7000 | loss: 1.5550
updated.... average c-index = 0.5539
itr: 8000 | loss: 1.8348
itr: 9000 | loss: 1.6144
itr: 10000 | loss: 2.2173
itr: 11000 | loss: 2.0753
itr: 12000 | loss: 1.7641


KeyboardInterrupt: 

In [19]:
### TRAINING AND TESTING IN LOOP:

In [23]:
nfolds = 10

for fold in range(nfolds):


    print('FOLD '+str(fold) + '...')
    
    ##### get training, testing, and validation data:
    df_train = df.loc[df['ID'].isin(trainingid_all.iloc[:,fold])]
    df_val = df.loc[df['ID'].isin(validationid_all.iloc[:,fold])]
    df_test = df.loc[df['ID'].isin(testingid_all.iloc[:,fold])]

    # ### TRAINING-TESTING SPLIT in the format suitable for this network

    (x_dim, x_dim_cont, x_dim_bin), (te_data, te_time, te_label), (te_mask1, te_mask2, te_mask3), (te_data_mi) = impt.import_dataset(df_ = df_test)
    (x_dim, x_dim_cont, x_dim_bin), (va_data, va_time, va_label), (va_mask1, va_mask2, va_mask3), (va_data_mi) = impt.import_dataset(df_ = df_val)
    (x_dim, x_dim_cont, x_dim_bin), (tr_data, tr_time, tr_label), (tr_mask1, tr_mask2, tr_mask3), (tr_data_mi) = impt.import_dataset(df_ = df_train)

    if boost_mode == 'ON':
        tr_data, tr_data_mi, tr_time, tr_label, tr_mask1, tr_mask2, tr_mask3 = f_get_boosted_trainset(tr_data, tr_data_mi, tr_time, tr_label, tr_mask1, tr_mask2, tr_mask3)  



        
        


    ##### CREATE AND TRAIN NETWORK:
    # tf.reset_default_graph()
    tf.compat.v1.reset_default_graph()

    # config = tf.ConfigProto()
    config = tf.compat.v1.ConfigProto()

    config.gpu_options.allow_growth = True
    sess = tf.compat.v1.Session(config=config)

    model = Model_Longitudinal_Attention(sess, "Dyanmic-DeepHit", input_dims, network_settings)
    # saver = tf.train.Saver()
    saver = tf.compat.v1.train.Saver()

    # sess.run(tf.global_variables_initializer())
    sess.run(tf.compat.v1.global_variables_initializer())

    ### TRAINING - BURN-IN
    if burn_in_mode == 'ON':
        print( "BURN-IN TRAINING ...")
        for itr in range(iteration_burn_in):
            x_mb, x_mi_mb, k_mb, t_mb, m1_mb, m2_mb, m3_mb = f_get_minibatch(mb_size, tr_data, tr_data_mi, tr_label, tr_time, tr_mask1, tr_mask2, tr_mask3)
            DATA = (x_mb, k_mb, t_mb)
            MISSING = (x_mi_mb)

            _, loss_curr = model.train_burn_in(DATA, MISSING, keep_prob, lr_train)

            if (itr+1)%1000 == 0:
                print('itr: {:04d} | loss: {:.4f}'.format(itr+1, loss_curr))


    ### TRAINING - MAIN
    print( "MAIN TRAINING ...")
    min_valid = 0.5

    for itr in range(iteration):
        x_mb, x_mi_mb, k_mb, t_mb, m1_mb, m2_mb, m3_mb = f_get_minibatch(mb_size, tr_data, tr_data_mi, tr_label, tr_time, tr_mask1, tr_mask2, tr_mask3)
        DATA = (x_mb, k_mb, t_mb)
        MASK = (m1_mb, m2_mb, m3_mb)
        MISSING = (x_mi_mb)
        PARAMETERS = (alpha, beta, gamma)

        _, loss_curr = model.train(DATA, MASK, MISSING, PARAMETERS, keep_prob, lr_train)

        if (itr+1)%1000 == 0:
            print('itr: {:04d} | loss: {:.4f}'.format(itr+1, loss_curr))

        ### VALIDATION  (based on average C-index of our interest)
        if (itr+1)%1000 == 0:        
            risk_all = f_get_risk_predictions(sess, model, va_data, va_data_mi, pred_time, eval_time)

            for p, p_time in enumerate(pred_time):
                pred_horizon = int(p_time)
                val_result1 = np.zeros([num_Event, len(eval_time)])

                for t, t_time in enumerate(eval_time):                
                    eval_horizon = int(t_time) + pred_horizon
                    for k in range(num_Event):
                        val_result1[k, t] = c_index(risk_all[k][:, p, t], va_time, (va_label[:,0] == k+1).astype(int), eval_horizon) #-1 for no event (not comparable)

                if p == 0:
                    val_final1 = val_result1
                else:
                    val_final1 = np.append(val_final1, val_result1, axis=0)

            tmp_valid = np.mean(val_final1)

            if tmp_valid >  min_valid:
                min_valid = tmp_valid
                saver.save(sess, file_path + '/model')
                print( 'updated.... average c-index = ' + str('%.4f' %(tmp_valid)))

                
                
                
         
        
        
                
    ### PREDICTION ON TEST SET               
    #saver.restore(sess, file_path + '/model')
 
    risk_all = f_get_risk_predictions(sess, model, te_data, te_data_mi, pred_time, eval_time)

    for p, p_time in enumerate(pred_time):
        pred_horizon = int(p_time)
        result1, result2 = np.zeros([num_Event, len(eval_time)]), np.zeros([num_Event, len(eval_time)])

        for t, t_time in enumerate(eval_time):                
            eval_horizon = int(t_time) + pred_horizon
            for k in range(num_Event):
                result1[k, t] = c_index(risk_all[k][:, p, t], te_time, (te_label[:,0] == k+1).astype(int), eval_horizon) #-1 for no event (not comparable)
                result2[k, t] = brier_score(risk_all[k][:, p, t], te_time, (te_label[:,0] == k+1).astype(int), eval_horizon) #-1 for no event (not comparable)

        if p == 0:
            final1, final2 = result1, result2
        else:
            final1, final2 = np.append(final1, result1, axis=0), np.append(final2, result2, axis=0)


            
            
    
    
    
    
    ### PRINT PERFORMANCE RESULTS
    row_header = []
    for p_time in pred_time:
        for t in range(num_Event):
            row_header.append('pred_time {}: event_{}'.format(p_time,k+1))

    col_header = []
    for t_time in eval_time:
        col_header.append('eval_time {}'.format(t_time))


    # c-index result
    df1 = pd.DataFrame(final1, index = row_header, columns=col_header)

    # brier-score result
    df2 = pd.DataFrame(final2, index = row_header, columns=col_header)

    print('========================================================')
    print('--------------------------------------------------------')
    print('- C-INDEX: ')
    print(df1)
    print('--------------------------------------------------------')
    print('- BRIER-SCORE: ')
    print(df2)
    print('========================================================')
    
    
    
    
    
    
    
    
    ### SAVE C-INDEX, BRIER SCORE, and PREDICTED PROB RISK ON TEST SET
    actual_fold = fold+1
    work_dir = 'U:/Hieu/CARDIA_longi_project'
    savedir = os.path.join(work_dir,'rdata_files/dynamic_deephit_ascvd_var_y15_fold_'+str(actual_fold)+'/')
    try: 
        os.makedirs(savedir)
    except OSError:
        if not os.path.isdir(savedir):
            raise



    c_over_time = df1.iloc[:,0]
    c_over_time.to_csv(savedir+'/c_index_tuned.csv', index = None, header = True)

    brier_over_time = df2.iloc[:,0]
    brier_over_time.to_csv(savedir+'/brier_score_tuned.csv', index = None, header = True)



    prob_risk_test_df = pd.DataFrame(risk_all[0][:,:,0])
    prob_risk_test_df.columns = pred_time
    prob_risk_test_df.insert(loc=0, column='ID', value=np.unique(df_test['ID']))
    prob_risk_test_df.to_csv(savedir+'/prob_risk_test_tuned.csv', index = None, header = True)



FOLD 0...
BURN-IN TRAINING ...
itr: 1000 | loss: 2.8584
itr: 2000 | loss: 1.5222
itr: 3000 | loss: 1.4856
MAIN TRAINING ...
itr: 1000 | loss: 1.6830
updated.... average c-index = 0.5393
itr: 2000 | loss: 2.1512
updated.... average c-index = 0.5687
itr: 3000 | loss: 1.6108
itr: 4000 | loss: 2.1840
itr: 5000 | loss: 1.5090
updated.... average c-index = 0.5795
itr: 6000 | loss: 1.0950
itr: 7000 | loss: 1.5397
updated.... average c-index = 0.5939
itr: 8000 | loss: 1.7656
itr: 9000 | loss: 1.7197
itr: 10000 | loss: 1.1308
itr: 11000 | loss: 1.0158
itr: 12000 | loss: 1.2947
itr: 13000 | loss: 2.0361
itr: 14000 | loss: 1.6770
itr: 15000 | loss: 1.8374
itr: 16000 | loss: 1.9535
itr: 17000 | loss: 1.3050
itr: 18000 | loss: 1.7168
itr: 19000 | loss: 1.2055
itr: 20000 | loss: 1.1645
itr: 21000 | loss: 1.2483
itr: 22000 | loss: 1.0601
itr: 23000 | loss: 1.8196
itr: 24000 | loss: 1.1338
itr: 25000 | loss: 1.0670
itr: 26000 | loss: 2.3029
itr: 27000 | loss: 2.1697
itr: 28000 | loss: 1.2295
itr: 2900

C:\Users\hnguye78\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\layers\legacy_rnn\rnn_cell_impl.py:903: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
C:\Users\hnguye78\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '
C:\Users\hnguye78\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1719: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and

BURN-IN TRAINING ...
itr: 1000 | loss: 2.8194
itr: 2000 | loss: 1.6141
itr: 3000 | loss: 1.5001
MAIN TRAINING ...
itr: 1000 | loss: 1.6779
updated.... average c-index = 0.5151
itr: 2000 | loss: 1.7990
updated.... average c-index = 0.5290
itr: 3000 | loss: 1.5998
itr: 4000 | loss: 1.1896
updated.... average c-index = 0.5379
itr: 5000 | loss: 1.3940
itr: 6000 | loss: 1.4818
updated.... average c-index = 0.5544
itr: 7000 | loss: 1.2846
itr: 8000 | loss: 1.4833
itr: 9000 | loss: 1.8510
updated.... average c-index = 0.5583
itr: 10000 | loss: 2.0098
itr: 11000 | loss: 1.8352
itr: 12000 | loss: 1.3408
itr: 13000 | loss: 1.3334
itr: 14000 | loss: 1.4397
itr: 15000 | loss: 1.5712
itr: 16000 | loss: 1.6061
itr: 17000 | loss: 1.0058
itr: 18000 | loss: 1.9418
itr: 19000 | loss: 1.2793
itr: 20000 | loss: 1.0496
itr: 21000 | loss: 1.9895
itr: 22000 | loss: 0.7993
itr: 23000 | loss: 1.2211
itr: 24000 | loss: 1.5958
itr: 25000 | loss: 1.3495
itr: 26000 | loss: 1.9995
itr: 27000 | loss: 1.5651
itr: 280

C:\Users\hnguye78\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\layers\legacy_rnn\rnn_cell_impl.py:903: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
C:\Users\hnguye78\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '
C:\Users\hnguye78\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1719: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and

BURN-IN TRAINING ...
itr: 1000 | loss: 2.2936
itr: 2000 | loss: 1.8271
itr: 3000 | loss: 1.9363
MAIN TRAINING ...
itr: 1000 | loss: 1.6016
updated.... average c-index = 0.5751
itr: 2000 | loss: 1.7979
updated.... average c-index = 0.5945
itr: 3000 | loss: 2.0241
updated.... average c-index = 0.6040
itr: 4000 | loss: 1.5603
itr: 5000 | loss: 1.3110
updated.... average c-index = 0.6069
itr: 6000 | loss: 1.3538
updated.... average c-index = 0.6124
itr: 7000 | loss: 1.6789
itr: 8000 | loss: 1.1816
itr: 9000 | loss: 0.8732
updated.... average c-index = 0.6158
itr: 10000 | loss: 1.9292
updated.... average c-index = 0.6177
itr: 11000 | loss: 1.5237
itr: 12000 | loss: 1.4024
itr: 13000 | loss: 1.0066
itr: 14000 | loss: 0.8884
updated.... average c-index = 0.6201
itr: 15000 | loss: 1.2418
itr: 16000 | loss: 1.0404
updated.... average c-index = 0.6237
itr: 17000 | loss: 1.7092
itr: 18000 | loss: 1.1628
updated.... average c-index = 0.6239
itr: 19000 | loss: 1.9254
updated.... average c-index = 0

...

BURN-IN TRAINING ...
itr: 1000 | loss: 2.4079
itr: 2000 | loss: 1.7935
itr: 3000 | loss: 1.4355
MAIN TRAINING ...
itr: 1000 | loss: 2.2183
updated.... average c-index = 0.6658
itr: 2000 | loss: 2.4965
updated.... average c-index = 0.6694
itr: 3000 | loss: 1.6028
updated.... average c-index = 0.6867
itr: 4000 | loss: 0.7728
updated.... average c-index = 0.6888
itr: 5000 | loss: 1.2184
updated.... average c-index = 0.6945
itr: 6000 | loss: 1.3758
updated.... average c-index = 0.7145
itr: 7000 | loss: 1.7682
itr: 8000 | loss: 1.1940
itr: 9000 | loss: 1.4874
updated.... average c-index = 0.7159
itr: 10000 | loss: 1.5649
updated.... average c-index = 0.7160
itr: 11000 | loss: 1.4442
itr: 12000 | loss: 1.7635
itr: 13000 | loss: 1.2996
itr: 14000 | loss: 1.2104
itr: 15000 | loss: 1.2353
updated.... average c-index = 0.7173
itr: 16000 | loss: 1.2422
itr: 17000 | loss: 0.9481
itr: 18000 | loss: 1.4457
itr: 19000 | loss: 1.4891
itr: 20000 | loss: 1.5013
itr: 21000 | loss: 1.2262
itr: 22000 | los

C:\Users\hnguye78\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\layers\legacy_rnn\rnn_cell_impl.py:903: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
C:\Users\hnguye78\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '
C:\Users\hnguye78\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1719: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and

BURN-IN TRAINING ...
itr: 1000 | loss: 2.5376
itr: 2000 | loss: 1.5836
itr: 3000 | loss: 1.7548
MAIN TRAINING ...
itr: 1000 | loss: 2.0923
updated.... average c-index = 0.5716
itr: 2000 | loss: 1.8664
itr: 3000 | loss: 1.1028
updated.... average c-index = 0.5729
itr: 4000 | loss: 2.3747
itr: 5000 | loss: 1.6851
updated.... average c-index = 0.5829
itr: 6000 | loss: 1.7217
itr: 7000 | loss: 1.0606
itr: 8000 | loss: 1.4443
itr: 9000 | loss: 0.9532
itr: 10000 | loss: 1.7020
itr: 11000 | loss: 1.2236
itr: 12000 | loss: 2.0894
itr: 13000 | loss: 0.9011
itr: 14000 | loss: 1.6963
itr: 15000 | loss: 0.9633
itr: 16000 | loss: 1.1804
itr: 17000 | loss: 1.4140
itr: 18000 | loss: 1.8526
itr: 19000 | loss: 1.1681
itr: 20000 | loss: 1.3412
itr: 21000 | loss: 1.1782
itr: 22000 | loss: 1.5851
itr: 23000 | loss: 1.4825
itr: 24000 | loss: 1.5611
itr: 25000 | loss: 1.2157
itr: 26000 | loss: 1.1792
itr: 27000 | loss: 0.9225
itr: 28000 | loss: 1.7069
itr: 29000 | loss: 1.3242
itr: 30000 | loss: 0.9356
itr:

C:\Users\hnguye78\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\layers\legacy_rnn\rnn_cell_impl.py:903: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
C:\Users\hnguye78\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '
C:\Users\hnguye78\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1719: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and

BURN-IN TRAINING ...
itr: 1000 | loss: 1.3788
itr: 2000 | loss: 1.0792
itr: 3000 | loss: 1.9168
MAIN TRAINING ...
itr: 1000 | loss: 1.4295
itr: 2000 | loss: 1.5902
updated.... average c-index = 0.5258
itr: 3000 | loss: 1.7788
updated.... average c-index = 0.5440
itr: 4000 | loss: 1.7009
updated.... average c-index = 0.5663
itr: 5000 | loss: 2.0533
itr: 6000 | loss: 2.9819
updated.... average c-index = 0.5696
itr: 7000 | loss: 1.8968
updated.... average c-index = 0.5706
itr: 8000 | loss: 1.2159
itr: 9000 | loss: 1.1522
itr: 10000 | loss: 2.5449
updated.... average c-index = 0.5772
itr: 11000 | loss: 1.0000
itr: 12000 | loss: 1.4795
updated.... average c-index = 0.5847
itr: 13000 | loss: 2.2270
updated.... average c-index = 0.5895
itr: 14000 | loss: 1.3292
itr: 15000 | loss: 1.2489
itr: 16000 | loss: 1.7968
itr: 17000 | loss: 1.4771
itr: 18000 | loss: 1.5807
itr: 19000 | loss: 1.2159
itr: 20000 | loss: 1.0190
itr: 21000 | loss: 0.9412
itr: 22000 | loss: 1.5830
itr: 23000 | loss: 1.9345
i

In [ ]:
# ##### CREATE DYNAMIC-DEEPFHT NETWORK
# # tf.reset_default_graph()
# tf.compat.v1.reset_default_graph()

# # config = tf.ConfigProto()
# config = tf.compat.v1.ConfigProto()

# config.gpu_options.allow_growth = True
# sess = tf.compat.v1.Session(config=config)

# model = Model_Longitudinal_Attention(sess, "Dyanmic-DeepHit", input_dims, network_settings)
# # saver = tf.train.Saver()
# saver = tf.compat.v1.train.Saver()

# # sess.run(tf.global_variables_initializer())
# sess.run(tf.compat.v1.global_variables_initializer())
 
# ### TRAINING - BURN-IN
# if burn_in_mode == 'ON':
#     print( "BURN-IN TRAINING ...")
#     for itr in range(iteration_burn_in):
#         x_mb, x_mi_mb, k_mb, t_mb, m1_mb, m2_mb, m3_mb = f_get_minibatch(mb_size, tr_data, tr_data_mi, tr_label, tr_time, tr_mask1, tr_mask2, tr_mask3)
#         DATA = (x_mb, k_mb, t_mb)
#         MISSING = (x_mi_mb)

#         _, loss_curr = model.train_burn_in(DATA, MISSING, keep_prob, lr_train)

#         if (itr+1)%1000 == 0:
#             print('itr: {:04d} | loss: {:.4f}'.format(itr+1, loss_curr))


# ### TRAINING - MAIN
# print( "MAIN TRAINING ...")
# min_valid = 0.5

# for itr in range(iteration):
#     x_mb, x_mi_mb, k_mb, t_mb, m1_mb, m2_mb, m3_mb = f_get_minibatch(mb_size, tr_data, tr_data_mi, tr_label, tr_time, tr_mask1, tr_mask2, tr_mask3)
#     DATA = (x_mb, k_mb, t_mb)
#     MASK = (m1_mb, m2_mb, m3_mb)
#     MISSING = (x_mi_mb)
#     PARAMETERS = (alpha, beta, gamma)

#     _, loss_curr = model.train(DATA, MASK, MISSING, PARAMETERS, keep_prob, lr_train)

#     if (itr+1)%1000 == 0:
#         print('itr: {:04d} | loss: {:.4f}'.format(itr+1, loss_curr))

#     ### VALIDATION  (based on average C-index of our interest)
#     if (itr+1)%1000 == 0:        
#         risk_all = f_get_risk_predictions(sess, model, va_data, va_data_mi, pred_time, eval_time)
        
#         for p, p_time in enumerate(pred_time):
#             pred_horizon = int(p_time)
#             val_result1 = np.zeros([num_Event, len(eval_time)])
            
#             for t, t_time in enumerate(eval_time):                
#                 eval_horizon = int(t_time) + pred_horizon
#                 for k in range(num_Event):
#                     val_result1[k, t] = c_index(risk_all[k][:, p, t], va_time, (va_label[:,0] == k+1).astype(int), eval_horizon) #-1 for no event (not comparable)
            
#             if p == 0:
#                 val_final1 = val_result1
#             else:
#                 val_final1 = np.append(val_final1, val_result1, axis=0)

#         tmp_valid = np.mean(val_final1)

#         if tmp_valid >  min_valid:
#             min_valid = tmp_valid
#             saver.save(sess, file_path + '/model')
#             print( 'updated.... average c-index = ' + str('%.4f' %(tmp_valid)))

In [12]:
fold

1

In [ ]:
                 
# risk_all = f_get_risk_predictions(sess, model, te_data, te_data_mi, pred_time, eval_time)

# for p, p_time in enumerate(pred_time):
#     pred_horizon = int(p_time)
#     result1, result2 = np.zeros([num_Event, len(eval_time)]), np.zeros([num_Event, len(eval_time)])

#     for t, t_time in enumerate(eval_time):                
#         eval_horizon = int(t_time) + pred_horizon
#         for k in range(num_Event):
#             result1[k, t] = c_index(risk_all[k][:, p, t], te_time, (te_label[:,0] == k+1).astype(int), eval_horizon) #-1 for no event (not comparable)
#             result2[k, t] = brier_score(risk_all[k][:, p, t], te_time, (te_label[:,0] == k+1).astype(int), eval_horizon) #-1 for no event (not comparable)
    
#     if p == 0:
#         final1, final2 = result1, result2
#     else:
#         final1, final2 = np.append(final1, result1, axis=0), np.append(final2, result2, axis=0)
        
        
        
# row_header = []
# for p_time in pred_time:
#     for t in range(num_Event):
#         row_header.append('pred_time {}: event_{}'.format(p_time,k+1))
            
# col_header = []
# for t_time in eval_time:
#     col_header.append('eval_time {}'.format(t_time))

# # c-index result
# df1 = pd.DataFrame(final1, index = row_header, columns=col_header)

# # brier-score result
# df2 = pd.DataFrame(final2, index = row_header, columns=col_header)

# ### PRINT RESULTS
# print('========================================================')
# print('--------------------------------------------------------')
# print('- C-INDEX: ')
# print(df1)
# print('--------------------------------------------------------')
# print('- BRIER-SCORE: ')
# print(df2)
# print('========================================================')



In [ ]:
# !pip install jupyter_contrib_nbextensions
# !jupyter contrib nbextension install --user
# !jupyter contrib nbextension install --sys-prefix
# !jupyter nbextension enable varInspector/main
